In [3]:
rm(list=ls())
library(cowplot)
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
library(scales)
library(repr)
library(stargazer)
options(repr.plot.width=10, repr.plot.height=10)
panel <- readRDS('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/data_v2.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/conley_selector_matrix.Rdata')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/social_adj_matrix.RData')
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/geo_adjacency_matrix.Rdata')
source('../utils/residualizer+genAlters.R')
source("../utils/labels_and_colors.r")
source('../utils/adjacency_and_cluster_robust_inference.R')
source('../utils/geo_utils.r')

In [4]:
panel %>%
  mutate(cbgv = sg_mcbgv) %>%
  mutate(asinh_sg_nhd = asinh(sg_nhd),
         log_sg_cbgv = log(cbgv),
         asinh_fb_rnstu = asinh(fb_rnstu)) %>% 
  mutate(asinh_sg_nhd.r = panel_residualizer(asinh_sg_nhd),
         log_sg_cbgv.r = panel_residualizer(log_sg_cbgv),
         asinh_fb_rnstu.r = panel_residualizer(asinh_fb_rnstu),
         sh_geo_alter.r = panel_residualizer(geo_alter_sh)) -> panel

[1]	train-rmse:0.251780+0.000236	test-rmse:0.251779+0.001090 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.128359+0.000164	test-rmse:0.128472+0.001255 
[3]	train-rmse:0.068629+0.000152	test-rmse:0.068735+0.001439 
[4]	train-rmse:0.041863+0.000184	test-rmse:0.041999+0.001447 
[5]	train-rmse:0.031698+0.000217	test-rmse:0.032003+0.001172 
[6]	train-rmse:0.028532+0.000226	test-rmse:0.028976+0.000884 
[7]	train-rmse:0.027617+0.000241	test-rmse:0.028123+0.000653 
[8]	train-rmse:0.027303+0.000248	test-rmse:0.027857+0.000614 
[9]	train-rmse:0.027189+0.000257	test-rmse:0.027806+0.000560 
[10]	train-rmse:0.027060+0.000232	test-rmse:0.027815+0.000555 
[11]	train-rmse:0.026987+0.000245	test-rmse:0.027819+0.000525 
[12]	train-rmse:0.026895+0.000265	test-rmse:0.027836+0.000503 
Stopping. Best iteration:
[9]	train-rmse:0.027189+0.000257	test-rmse:0.027806+0.000560

[1]	train-rmse:0.256279+0.000267	t

In [5]:
# DiD - focal state only
DiD.sp_fbrstu <- felm(asinh_fb_rnstu.r ~ 0 + stay_home.r + ban_gmr.r | county_fips + ds | 0 | 
                      0, panel, weights = panel$n, keepCX = T)
out_fbrstu <- summarize.felm.fit.with.adj(fm = DiD.sp_fbrstu, conley.selector.matrix, panel$state_abbv)

DiD.sp_hd <- felm(asinh_sg_nhd.r ~ 0 + stay_home.r + ban_gmr.r | county_fips + ds | 0 | 0, 
                  panel, weights = panel$n, keepCX = T)
out_sp_hd <- summarize.felm.fit.with.adj(fm = DiD.sp_hd, conley.selector.matrix, panel$state_abbv)

DiD.sp_cbgv <- felm(log_sg_cbgv.r ~ 0 + stay_home.r + ban_gmr.r | county_fips + ds | 0 | 0, 
                    panel, weights = panel$n, keepCX = T)
out_sp_cbgv <- summarize.felm.fit.with.adj(fm = DiD.sp_cbgv, conley.selector.matrix, panel$state_abbv)

DiD.sp_btvrc <- felm(fb_btvrc.r ~ 0 + stay_home.r + ban_gmr.r | county_fips + ds | 0 | 0, 
                     panel, weights = panel$n, keepCX = T)
out_sp_btvrc <- summarize.felm.fit.with.adj(fm = DiD.sp_btvrc, conley.selector.matrix, panel$state_abbv)

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [6]:
# DiD - alter policy ITT just geo
DID.mp_go.aITT.sg_hd <- felm(asinh_sg_nhd.r ~ 0 + stay_home.r + geo_alter_sh.r 
                             + ban_gmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_hd_aitt_sg_go <- summarize.felm.fit.with.adj(fm = DID.mp_go.aITT.sg_hd, 
                                                 conley.selector.matrix, panel$state_abbv)

DID.mp_go.aITT.sg_rstu <- felm(asinh_fb_rnstu.r ~ 0 + stay_home.r + geo_alter_sh.r 
                             + ban_gmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_rstu_aitt_sg_go <- summarize.felm.fit.with.adj(fm = DID.mp_go.aITT.sg_rstu, 
                                                   conley.selector.matrix, panel$state_abbv)

DID.mp_go.aITT.sg_cbgv <- felm(log_sg_cbgv.r ~ 0 + stay_home.r + geo_alter_sh.r 
                             + ban_gmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_cbgv_aitt_sg_go <- summarize.felm.fit.with.adj(fm = DID.mp_go.aITT.sg_cbgv, 
                                                   conley.selector.matrix, panel$state_abbv)

DID.mp_go.aITT.sg_btvrc <- felm(fb_btvrc.r ~ 0 + stay_home.r + geo_alter_sh.r 
                             + ban_gmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_btvrc_aitt_sg_go <- summarize.felm.fit.with.adj(fm = DID.mp_go.aITT.sg_btvrc, 
                                                    conley.selector.matrix, panel$state_abbv)

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [7]:
# DiD - alter policy ITT social and geo
DID.mp.aITT.sg_hd <- felm(asinh_sg_nhd.r ~ 0 + stay_home.r + alter_sh.r + geo_alter_sh.r 
                          + ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r  | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_hd_aitt_sg <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_hd, conley.selector.matrix, panel$state_abbv)

DID.mp.aITT.sg_rstu <- felm(asinh_fb_rnstu.r ~ 0 + stay_home.r + alter_sh.r + geo_alter_sh.r 
                          + ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_rstu_aitt_sg <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_rstu, conley.selector.matrix, panel$state_abbv)

DID.mp.aITT.sg_cbgv <- felm(log_sg_cbgv.r ~ 0 + stay_home.r + alter_sh.r + geo_alter_sh.r 
                          + ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_cbgv_aitt_sg <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_cbgv, conley.selector.matrix, panel$state_abbv)

DID.mp.aITT.sg_btvrc <- felm(fb_btvrc.r ~ 0 + stay_home.r + alter_sh.r + geo_alter_sh.r 
                          + ban_gmr.r + alter_bgmr.r + geo_alter_bgmr.r | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_btvrc_aitt_sg <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_btvrc, conley.selector.matrix, panel$state_abbv)

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [8]:
# DiD - alter policy ITT social and geo with interactions
DID.mp.aITT.sg_hd_int <- felm(asinh_sg_nhd.r ~ 0 +  ban_gmr.r*(alter_bgmr.r + geo_alter_bgmr.r) +
                          stay_home.r*(alter_sh.r + geo_alter_sh.r) | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_hd_aitt_sg_int <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_hd_int, conley.selector.matrix, panel$state_abbv)

DID.mp.aITT.sg_rstu_int <- felm(asinh_fb_rnstu.r ~ 0 +  ban_gmr.r*(alter_bgmr.r + geo_alter_bgmr.r) +
                          stay_home.r*(alter_sh.r + geo_alter_sh.r) | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_rstu_aitt_sg_int <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_rstu_int, conley.selector.matrix, panel$state_abbv)

DID.mp.aITT.sg_cbgv_int <- felm(log_sg_cbgv.r ~ 0 +  ban_gmr.r*(alter_bgmr.r + geo_alter_bgmr.r) +
                          stay_home.r*(alter_sh.r + geo_alter_sh.r) | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_cbgv_aitt_sg_int <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_cbgv_int, conley.selector.matrix, panel$state_abbv)

DID.mp.aITT.sg_btvrc_int <- felm(fb_btvrc.r ~ 0 +  ban_gmr.r*(alter_bgmr.r + geo_alter_bgmr.r) +
                          stay_home.r*(alter_sh.r + geo_alter_sh.r) | county_fips + ds | 0 | state_abbv, 
                          panel, keepCX = T, weights = panel$n)
out_btvrc_aitt_sg_int <- summarize.felm.fit.with.adj(fm = DID.mp.aITT.sg_btvrc_int, conley.selector.matrix, panel$state_abbv)

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [9]:
# Split out in-state and out-of-state alters
same_state_matrix = do.call(rbind, lapply(
    colnames(WM),
    function(i) state_code_from_fips(i) == state_code_from_fips(colnames(WM))
    ))

# split social matrix
WM_same_state <- same_state_matrix * WM
WM_diff_state <- WM - WM_same_state

# Renormalize social matrices
WM_same_state <- WM_same_state/rowSums(WM_same_state) 
WM_diff_state <- WM_diff_state/rowSums(WM_diff_state) 

# Split geo matrix
gWM_same_state <- same_state_matrix * gWM
gWM_diff_state <- gWM - gWM_same_state

# Renormalize geo matrices
gWM_same_state <- gWM_same_state/rowSums(gWM_same_state) 
gWM_diff_state <- gWM_diff_state/rowSums(gWM_diff_state) 

In [10]:
weightedAlters <- function(wm, colname = NA, df = df, ...) {
    df %>% 
        select(ds, county_fips, ...) %>%
        spread(key = county_fips, value = ...) %>%
        ungroup() %>%
        select(-ds) %>%
        as.matrix() -> txn_data
    
    df %>% 
      ungroup() %>%
      select(ds) %>%
      distinct() %>%
      arrange(ds) -> dates
    
    outMatrix <- tcrossprod(txn_data, wm)
    colnames(outMatrix) <- colnames(txn_data)
    
    data.frame(dates, outMatrix) %>%
        gather(key = 'county_fips', value = 'value', -ds) %>%
        mutate(county_fips = as.integer(str_sub(county_fips, 2, -1))) %>% 
        arrange(ds, county_fips) %>%
        select(-ds, -county_fips) -> out_df
    if (is.null(colname)) {
        return(out_df[, 1])
    } else {
        colnames(out_df)[1] <- colname
        return(out_df)
    }
}

panel$alter_same_state <- weightedAlters(WM_same_state, NULL, ones, df = panel %>% mutate(ones = 1))
panel$alter_ban_gmr_same_state <- weightedAlters(WM_same_state, NULL, ban_gmr, df = panel)
panel$alter_ban_gmr_diff_state <- weightedAlters(WM_diff_state, NULL, ban_gmr, df = panel)
panel$alter_sh_same_state <- weightedAlters(WM_same_state, NULL, stay_home, df = panel)
panel$alter_sh_diff_state <- weightedAlters(WM_diff_state, NULL, stay_home, df = panel)

panel$geo_alter_same_state <- weightedAlters(gWM_same_state, NULL, ones, df = panel %>% mutate(ones = 1))
panel$geo_alter_ban_gmr_same_state <- weightedAlters(gWM_same_state, NULL, ban_gmr, df = panel)
panel$geo_alter_ban_gmr_diff_state <- weightedAlters(gWM_diff_state, NULL, ban_gmr, df = panel)
panel$geo_alter_sh_same_state <- weightedAlters(gWM_same_state, NULL, stay_home, df = panel)
panel$geo_alter_sh_diff_state <- weightedAlters(gWM_diff_state, NULL, stay_home, df = panel)

# Fill in DC same state
panel %>% 
  mutate(alter_same_state = ifelse(is.na(alter_same_state), 1, alter_same_state),
         alter_ban_gmr_same_state = ifelse(is.na(alter_ban_gmr_same_state), ban_gmr, alter_ban_gmr_same_state),
         alter_sh_same_state = ifelse(is.na(alter_sh_same_state), stay_home, alter_sh_same_state),
         geo_alter_same_state = ifelse(is.na(geo_alter_same_state), 1, geo_alter_same_state),
         geo_alter_ban_gmr_same_state = ifelse(is.na(geo_alter_ban_gmr_same_state), 
                                               ban_gmr, geo_alter_ban_gmr_same_state),
         geo_alter_sh_same_state = ifelse(is.na(geo_alter_sh_same_state),
                                         stay_home, geo_alter_sh_same_state)
         ) -> panel

In [11]:
panel %>% 
  mutate(alter_same_state.r = panel_residualizer(alter_same_state),
         alter_ban_gmr_same_state.r = panel_residualizer(alter_ban_gmr_same_state),
         alter_ban_gmr_diff_state.r = panel_residualizer(alter_ban_gmr_diff_state),
         alter_sh_same_state.r = panel_residualizer(alter_sh_same_state),
         alter_sh_diff_state.r = panel_residualizer(alter_sh_diff_state),
         geo_alter_same_state.r = panel_residualizer(geo_alter_same_state),
         geo_alter_ban_gmr_same_state.r = panel_residualizer(geo_alter_ban_gmr_same_state),
         geo_alter_ban_gmr_diff_state.r = panel_residualizer(geo_alter_ban_gmr_diff_state),
         geo_alter_sh_same_state.r = panel_residualizer(geo_alter_sh_same_state),
         geo_alter_sh_diff_state.r = panel_residualizer(geo_alter_sh_diff_state)) -> panel

[1]	train-rmse:0.250152+0.000000	test-rmse:0.250151+0.000000 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.125151+0.000000	test-rmse:0.125150+0.000000 
[3]	train-rmse:0.062612+0.000000	test-rmse:0.062612+0.000000 
[4]	train-rmse:0.031325+0.000000	test-rmse:0.031325+0.000000 
[5]	train-rmse:0.015672+0.000000	test-rmse:0.015672+0.000000 
[6]	train-rmse:0.007841+0.000000	test-rmse:0.007841+0.000000 
[7]	train-rmse:0.003923+0.000000	test-rmse:0.003923+0.000000 
[8]	train-rmse:0.001963+0.000000	test-rmse:0.001963+0.000000 
[9]	train-rmse:0.000982+0.000000	test-rmse:0.000982+0.000000 
[10]	train-rmse:0.000491+0.000000	test-rmse:0.000491+0.000000 
[11]	train-rmse:0.000246+0.000000	test-rmse:0.000246+0.000000 
[12]	train-rmse:0.000123+0.000000	test-rmse:0.000123+0.000000 
[13]	train-rmse:0.000062+0.000000	test-rmse:0.000062+0.000000 
[14]	train-rmse:0.000031+0.000000	test-rmse:0.000031+0.0000

In [12]:
# DiD - alter policy ITT just geo (split in- and out of state)
did_sgnhd_geo_split <-  felm(asinh_sg_nhd.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_sgnhd_geo_split <- summarize.felm.fit.with.adj(fm = did_sgnhd_geo_split, 
                                                 conley.selector.matrix, panel$state_abbv)

did_fbrnstu_geo_split <-  felm(asinh_fb_rnstu.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_fbrnstu_geo_split <- summarize.felm.fit.with.adj(fm = did_fbrnstu_geo_split, 
                                                 conley.selector.matrix, panel$state_abbv)

did_sgcbgv_geo_split <-  felm(log_sg_cbgv.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_sgcbgv_geo_split <- summarize.felm.fit.with.adj(fm = did_sgcbgv_geo_split, 
                                                 conley.selector.matrix, panel$state_abbv)

did_btvrc_geo_split <-  felm(fb_btvrc.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_btvrc_geo_split <- summarize.felm.fit.with.adj(fm = did_btvrc_geo_split, 
                                                 conley.selector.matrix, panel$state_abbv)

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [13]:
did_sgnhd_both_split <-  felm(asinh_sg_nhd.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                              alter_sh_same_state.r + alter_sh_diff_state.r +
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r + 
                              alter_ban_gmr_same_state.r + alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_sgnhd_both_split <- summarize.felm.fit.with.adj(fm = did_sgnhd_both_split, 
                                                 conley.selector.matrix, panel$state_abbv)

did_fbrnstu_both_split <-  felm(asinh_fb_rnstu.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                              alter_sh_same_state.r + alter_sh_diff_state.r +
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r + 
                              alter_ban_gmr_same_state.r + alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_fbrnstu_both_split <- summarize.felm.fit.with.adj(fm = did_fbrnstu_both_split, 
                                                 conley.selector.matrix, panel$state_abbv)

did_sgcbgv_both_split <-  felm(log_sg_cbgv.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                              alter_sh_same_state.r + alter_sh_diff_state.r +
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r + 
                              alter_ban_gmr_same_state.r + alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_sgcbgv_both_split <- summarize.felm.fit.with.adj(fm = did_sgcbgv_both_split, 
                                                 conley.selector.matrix, panel$state_abbv)

did_btvrc_both_split <-  felm(fb_btvrc.r ~ 0 + 
                             stay_home.r + geo_alter_sh_same_state.r +  geo_alter_sh_diff_state.r + 
                              alter_sh_same_state.r + alter_sh_diff_state.r +
                             + ban_gmr.r + geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_diff_state.r + 
                              alter_ban_gmr_same_state.r + alter_ban_gmr_diff_state.r | 
                             county_fips + ds | 0 | 0, 
                          panel, keepCX = T, weights = panel$n)
out_btvrc_both_split <- summarize.felm.fit.with.adj(fm = did_btvrc_both_split, 
                                                 conley.selector.matrix, panel$state_abbv)

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [14]:
model_save_location = '/nfs/sloanlab004/projects/covid_mobility_proj/models'

# Save focal only models
# The first four rows here are backwards, but this is also backwards in the load file. 
#Should fix both (and remove both comments!)
saveRDS(DiD.sp_fbrstu, file=str_c(model_save_location, 'did_sgnhd_focal.rds'))
saveRDS(out_fbrstu, file=str_c(model_save_location, 'did_sgnhd_focal_vcov.rds'))
saveRDS(DiD.sp_hd, file=str_c(model_save_location, 'did_fbnrstu_focal.rds'))
saveRDS(out_sp_hd, file=str_c(model_save_location, 'did_fbrnstu_focal_vcov.rds'))
saveRDS(DiD.sp_cbgv, file=str_c(model_save_location, 'did_sgcbgv_focal.rds'))
saveRDS(out_sp_cbgv, file=str_c(model_save_location, 'did_sgcbgv_focal_vcov.rds'))
saveRDS(DiD.sp_btvrc, file=str_c(model_save_location, 'did_fbbtvrc_focal.rds'))
saveRDS(out_sp_btvrc, file=str_c(model_save_location, 'did_fbbtvrc_focal_vcov.rds'))

# Save focal + geo models
saveRDS(DID.mp_go.aITT.sg_hd, file=str_c(model_save_location, 'did_sgnhd_geo.rds'))
saveRDS(out_hd_aitt_sg_go, file=str_c(model_save_location, 'did_sgnhd_geo_vcov.rds'))
saveRDS(DID.mp_go.aITT.sg_rstu, file=str_c(model_save_location, 'did_fbnrstu_geo.rds'))
saveRDS(out_rstu_aitt_sg_go, file=str_c(model_save_location, 'did_fbrnstu_geo_vcov.rds'))
saveRDS(DID.mp_go.aITT.sg_cbgv, file=str_c(model_save_location, 'did_sgcbgv_geo.rds'))
saveRDS(out_cbgv_aitt_sg_go, file=str_c(model_save_location, 'did_sgcbgv_geo_vcov.rds'))
saveRDS(DID.mp_go.aITT.sg_btvrc, file=str_c(model_save_location, 'did_fbbtvrc_geo.rds'))
saveRDS(out_btvrc_aitt_sg_go, file=str_c(model_save_location, 'did_fbbtvrc_geo_vcov.rds'))

# Save focal + geo + social models
saveRDS(DID.mp.aITT.sg_hd, file=str_c(model_save_location, 'did_sgnhd_both.rds'))  
saveRDS(out_hd_aitt_sg, file=str_c(model_save_location, 'did_sgnhd_both_vcov.rds'))
saveRDS(DID.mp.aITT.sg_rstu, file=str_c(model_save_location, 'did_fbnrstu_both.rds'))                                      
saveRDS(out_rstu_aitt_sg, file=str_c(model_save_location, 'did_fbrnstu_both_vcov.rds'))
saveRDS(DID.mp.aITT.sg_cbgv, file=str_c(model_save_location, 'did_sgcbgv_both.rds'))                 
saveRDS(out_cbgv_aitt_sg, file=str_c(model_save_location, 'did_sgcbgv_both_vcov.rds'))
saveRDS(DID.mp.aITT.sg_btvrc, file=str_c(model_save_location, 'did_fbbtvrc_both.rds'))          
saveRDS(out_btvrc_aitt_sg, file=str_c(model_save_location, 'did_fbbtvrc_both_vcov.rds'))

# Save focal + geo + social with interaction models
saveRDS(DID.mp.aITT.sg_hd_int, file=str_c(model_save_location, 'did_sgnhd_both_int.rds'))                          
saveRDS(out_hd_aitt_sg_int, file=str_c(model_save_location, 'did_sgnhd_both_int_vcov.rds'))
saveRDS(DID.mp.aITT.sg_rstu_int, file=str_c(model_save_location, 'did_fbnrstu_both_int.rds'))        
saveRDS(out_rstu_aitt_sg_int, file=str_c(model_save_location, 'did_fbrnstu_both_int_vcov.rds'))
saveRDS(DID.mp.aITT.sg_cbgv_int, file=str_c(model_save_location, 'did_sgcbgv_both_int.rds'))      
saveRDS(out_cbgv_aitt_sg_int, file=str_c(model_save_location, 'did_sgcbgv_both_int_vcov.rds'))
saveRDS(DID.mp.aITT.sg_btvrc_int, file=str_c(model_save_location, 'did_fbbtvrc_both_int.rds'))      
saveRDS(out_btvrc_aitt_sg_int, file=str_c(model_save_location, 'did_fbbtvrc_both_int_vcov.rds'))

# Save focal + geo with split in-state and out-of-state models
saveRDS(did_sgnhd_geo_split, file=str_c(model_save_location, 'did_sgnhd_geo_split.rds'))                          
saveRDS(out_sgnhd_geo_split, file=str_c(model_save_location, 'did_sgnhd_geo_split_vcov.rds'))
saveRDS(did_fbrnstu_geo_split, file=str_c(model_save_location, 'did_fbnrstu_geo_split.rds'))                          
saveRDS(out_fbrnstu_geo_split, file=str_c(model_save_location, 'did_fbrnstu_geo_split_vcov.rds'))
saveRDS(did_sgcbgv_geo_split, file=str_c(model_save_location, 'did_sgcbgv_geo_split.rds'))     
saveRDS(out_sgcbgv_geo_split, file=str_c(model_save_location, 'did_sgcbgv_geo_split_vcov.rds'))
saveRDS(did_btvrc_geo_split, file=str_c(model_save_location, 'did_fbbtvrc_geo_split.rds'))      
saveRDS(out_btvrc_geo_split, file=str_c(model_save_location, 'did_fbbtvrc_geo_split_vcov.rds'))

# Save focal + geo + social with split in-state and out-of-state models
saveRDS(did_sgnhd_both_split, file=str_c(model_save_location, 'did_sgnhd_both_split.rds'))                          
saveRDS(out_sgnhd_both_split, file=str_c(model_save_location, 'did_sgnhd_both_split_vcov.rds'))
saveRDS(did_fbrnstu_both_split, file=str_c(model_save_location, 'did_fbnrstu_both_split.rds'))      
saveRDS(out_fbrnstu_both_split, file=str_c(model_save_location, 'did_fbrnstu_both_split_vcov.rds'))
saveRDS(did_sgcbgv_both_split, file=str_c(model_save_location, 'did_sgcbgv_both_split.rds'))    
saveRDS(out_sgcbgv_both_split, file=str_c(model_save_location, 'did_sgcbgv_both_split_vcov.rds'))
saveRDS(did_btvrc_both_split, file=str_c(model_save_location, 'did_fbbtvrc_both_split.rds'))     
saveRDS(out_btvrc_both_split, file=str_c(model_save_location, 'did_fbbtvrc_both_split_vcov.rds'))

In [15]:
## Analysis that looks at leads and lags for pre-trends + heterogeneous effects after roll out

In [16]:
panel %>% 
  group_by(county_fips) %>% 
  arrange(ds) %>% 
  mutate(# Self leads and lags
         stay_home_m1.r = lag(stay_home.r, 1),
         stay_home_m2.r = lag(stay_home.r, 2),
         stay_home_m3.r = lag(stay_home.r, 3),
         stay_home_p1.r = lead(stay_home.r, 1),
         stay_home_p2.r = lead(stay_home.r, 2),
         stay_home_p3.r = lead(stay_home.r, 3),
         ban_gmr_m1.r = lag(ban_gmr.r, 1),
         ban_gmr_m2.r = lag(ban_gmr.r, 2),
         ban_gmr_m3.r = lag(ban_gmr.r, 3),
         ban_gmr_p1.r = lead(ban_gmr.r, 1),
         ban_gmr_p2.r = lead(ban_gmr.r, 2),
         ban_gmr_p3.r = lead(ban_gmr.r, 3),
         # Geo alter leads and lags
         geo_alter_sh_m1.r = lag(geo_alter_sh.r, 1),
         geo_alter_sh_m2.r = lag(geo_alter_sh.r, 2),
         geo_alter_sh_m3.r = lag(geo_alter_sh.r, 3),
         geo_alter_sh_p1.r = lead(geo_alter_sh.r, 1),
         geo_alter_sh_p2.r = lead(geo_alter_sh.r, 2),
         geo_alter_sh_p3.r = lead(geo_alter_sh.r, 3),
         geo_alter_bgmr_m1.r = lag(geo_alter_bgmr.r, 1),
         geo_alter_bgmr_m2.r = lag(geo_alter_bgmr.r, 2),
         geo_alter_bgmr_m3.r = lag(geo_alter_bgmr.r, 3),
         geo_alter_bgmr_p1.r = lead(geo_alter_bgmr.r, 1),
         geo_alter_bgmr_p2.r = lead(geo_alter_bgmr.r, 2),
         geo_alter_bgmr_p3.r = lead(geo_alter_bgmr.r, 3),
         # Self alter leads and lags
         alter_sh_m1.r = lag(alter_sh.r, 1),
         alter_sh_m2.r = lag(alter_sh.r, 2),
         alter_sh_m3.r = lag(alter_sh.r, 3),
         alter_sh_p1.r = lead(alter_sh.r, 1),
         alter_sh_p2.r = lead(alter_sh.r, 2),
         alter_sh_p3.r = lead(alter_sh.r, 3),
         alter_bgmr_m1.r = lag(alter_bgmr.r, 1),
         alter_bgmr_m2.r = lag(alter_bgmr.r, 2),
         alter_bgmr_m3.r = lag(alter_bgmr.r, 3),
         alter_bgmr_p1.r = lead(alter_bgmr.r, 1), 
         alter_bgmr_p2.r = lead(alter_bgmr.r, 2),
         alter_bgmr_p3.r = lead(alter_bgmr.r, 3),
         # Geo alter leads and lags (within state)
         geo_alter_sh_same_state_m1.r = lag(geo_alter_sh_same_state.r, 1),
         geo_alter_sh_same_state_m2.r = lag(geo_alter_sh_same_state.r, 2),
         geo_alter_sh_same_state_m3.r = lag(geo_alter_sh_same_state.r, 3),
         geo_alter_sh_same_state_p1.r = lead(geo_alter_sh_same_state.r, 1),
         geo_alter_sh_same_state_p2.r = lead(geo_alter_sh_same_state.r, 2),
         geo_alter_sh_same_state_p3.r = lead(geo_alter_sh_same_state.r, 3),
         geo_alter_ban_gmr_same_state_m1.r = lag(geo_alter_ban_gmr_same_state.r, 1),
         geo_alter_ban_gmr_same_state_m2.r = lag(geo_alter_ban_gmr_same_state.r, 2),
         geo_alter_ban_gmr_same_state_m3.r = lag(geo_alter_ban_gmr_same_state.r, 3),
         geo_alter_ban_gmr_same_state_p1.r = lead(geo_alter_ban_gmr_same_state.r, 1),
         geo_alter_ban_gmr_same_state_p2.r = lead(geo_alter_ban_gmr_same_state.r, 2),
         geo_alter_ban_gmr_same_state_p3.r = lead(geo_alter_ban_gmr_same_state.r, 3),
         # Geo alter leads and lags (out of state)
         geo_alter_sh_diff_state_m1.r = lag(geo_alter_sh_diff_state.r, 1),
         geo_alter_sh_diff_state_m2.r = lag(geo_alter_sh_diff_state.r, 2),
         geo_alter_sh_diff_state_m3.r = lag(geo_alter_sh_diff_state.r, 3),
         geo_alter_sh_diff_state_p1.r = lead(geo_alter_sh_diff_state.r, 1),
         geo_alter_sh_diff_state_p2.r = lead(geo_alter_sh_diff_state.r, 2),
         geo_alter_sh_diff_state_p3.r = lead(geo_alter_sh_diff_state.r, 3),
         geo_alter_ban_gmr_diff_state_m1.r = lag(geo_alter_ban_gmr_diff_state.r, 1),
         geo_alter_ban_gmr_diff_state_m2.r = lag(geo_alter_ban_gmr_diff_state.r, 2),
         geo_alter_ban_gmr_diff_state_m3.r = lag(geo_alter_ban_gmr_diff_state.r, 3),
         geo_alter_ban_gmr_diff_state_p1.r = lead(geo_alter_ban_gmr_diff_state.r, 1),
         geo_alter_ban_gmr_diff_state_p2.r = lead(geo_alter_ban_gmr_diff_state.r, 2),
         geo_alter_ban_gmr_diff_state_p3.r = lead(geo_alter_ban_gmr_diff_state.r, 3),
         # Social alter leads and lags (same state)
         alter_sh_same_state_m1.r = lag(alter_sh_same_state.r, 1),
         alter_sh_same_state_m2.r = lag(alter_sh_same_state.r, 2),
         alter_sh_same_state_m3.r = lag(alter_sh_same_state.r, 3),
         alter_sh_same_state_p1.r = lead(alter_sh_same_state.r, 1),
         alter_sh_same_state_p2.r = lead(alter_sh_same_state.r, 2),
         alter_sh_same_state_p3.r = lead(alter_sh_same_state.r, 3),
         alter_ban_gmr_same_state_m1.r = lag(alter_ban_gmr_same_state.r, 1),
         alter_ban_gmr_same_state_m2.r = lag(alter_ban_gmr_same_state.r, 2),
         alter_ban_gmr_same_state_m3.r = lag(alter_ban_gmr_same_state.r, 3),
         alter_ban_gmr_same_state_p1.r = lead(alter_ban_gmr_same_state.r, 1),
         alter_ban_gmr_same_state_p2.r = lead(alter_ban_gmr_same_state.r, 2),
         alter_ban_gmr_same_state_p3.r = lead(alter_ban_gmr_same_state.r, 3),      
         # Social alter leads and lags (out of state)
         alter_sh_diff_state_m1.r = lag(alter_sh_diff_state.r, 1),
         alter_sh_diff_state_m2.r = lag(alter_sh_diff_state.r, 2),
         alter_sh_diff_state_m3.r = lag(alter_sh_diff_state.r, 3),
         alter_sh_diff_state_p1.r = lead(alter_sh_diff_state.r, 1),
         alter_sh_diff_state_p2.r = lead(alter_sh_diff_state.r, 2),
         alter_sh_diff_state_p3.r = lead(alter_sh_diff_state.r, 3),
         alter_ban_gmr_diff_state_m1.r = lag(alter_ban_gmr_diff_state.r, 1),
         alter_ban_gmr_diff_state_m2.r = lag(alter_ban_gmr_diff_state.r, 2),
         alter_ban_gmr_diff_state_m3.r = lag(alter_ban_gmr_diff_state.r, 3),
         alter_ban_gmr_diff_state_p1.r = lead(alter_ban_gmr_diff_state.r, 1),
         alter_ban_gmr_diff_state_p2.r = lead(alter_ban_gmr_diff_state.r, 2),
         alter_ban_gmr_diff_state_p3.r = lead(alter_ban_gmr_diff_state.r, 3)            
  ) %>%
  ungroup() %>% 
  arrange(ds, county_fips) %>% 
  filter(!is.na(stay_home_m3.r) & !is.na(stay_home_p3.r)) -> panel_ll

In [17]:
# DiD - focal state only (with leads and lags)
ll_DiD.sp_fbrstu <- felm(asinh_fb_rnstu.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r
                         | county_fips + ds | 0 | 
                      0, panel_ll, weights = panel_ll$n, keepCX = T)
ll_out_fbrstu <- summarize.felm.fit.with.adj(fm = ll_DiD.sp_fbrstu, conley.selector.matrix, 
                                             panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_DiD.sp_hd <- felm(asinh_sg_nhd.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r | county_fips + ds | 0 | 0, 
                  panel_ll, weights = panel_ll$n, keepCX = T)
ll_out_sp_hd <- summarize.felm.fit.with.adj(fm = ll_DiD.sp_hd, conley.selector.matrix, 
                                            panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_DiD.sp_cbgv <- felm(log_sg_cbgv.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r | county_fips + ds | 0 | 0, 
                    panel_ll, weights = panel_ll$n, keepCX = T)
ll_out_sp_cbgv <- summarize.felm.fit.with.adj(fm = ll_DiD.sp_cbgv, conley.selector.matrix, 
                                              panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_DiD.sp_btvrc <- felm(fb_btvrc.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r | county_fips + ds | 0 | 0, 
                     panel_ll, weights = panel_ll$n, keepCX = T)
ll_out_sp_btvrc <- summarize.felm.fit.with.adj(fm = ll_DiD.sp_btvrc, conley.selector.matrix, 
                                               panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [18]:
# DiD - alter policy ITT just geo (with leads and lags)
ll_DID.mp_go.aITT.sg_hd <- felm(asinh_sg_nhd.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                                geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                                geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                                geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                                geo_alter_bgmr_m3.r | county_fips + ds | 0 | state_abbv, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_hd_aitt_sg_go <- summarize.felm.fit.with.adj(fm = ll_DID.mp_go.aITT.sg_hd, 
                                                 conley.selector.matrix, panel_ll$state_abbv,
                                                   n_ds=length(unique(panel_ll$ds)))

ll_DID.mp_go.aITT.sg_rstu <- felm(asinh_fb_rnstu.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                                geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                                geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                                geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                                geo_alter_bgmr_m3.r | county_fips + ds | 0 | state_abbv, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_rstu_aitt_sg_go <- summarize.felm.fit.with.adj(fm = ll_DID.mp_go.aITT.sg_rstu, 
                                                   conley.selector.matrix, panel_ll$state_abbv,
                                                     n_ds=length(unique(panel_ll$ds)))

ll_DID.mp_go.aITT.sg_cbgv <- felm(log_sg_cbgv.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                                geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                                geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                                geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                                geo_alter_bgmr_m3.r | county_fips + ds | 0 | state_abbv, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_cbgv_aitt_sg_go <- summarize.felm.fit.with.adj(fm = ll_DID.mp_go.aITT.sg_cbgv, 
                                                   conley.selector.matrix, panel_ll$state_abbv,
                                                     n_ds=length(unique(panel_ll$ds)))

ll_DID.mp_go.aITT.sg_btvrc <- felm(fb_btvrc.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                         stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                         stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                         ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                                geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                                geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                                geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                                geo_alter_bgmr_m3.r | county_fips + ds | 0 | state_abbv, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_btvrc_aitt_sg_go <- summarize.felm.fit.with.adj(fm = ll_DID.mp_go.aITT.sg_btvrc, 
                                                    conley.selector.matrix, panel_ll$state_abbv,
                                                      n_ds=length(unique(panel_ll$ds)))

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [19]:
# DiD - alter policy ITT social and geo (with leads and lags)
ll_DID.mp.aITT.sg_hd <- felm(asinh_sg_nhd.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                             stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                             stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                             ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                             geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                             geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                             geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                             geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                             geo_alter_bgmr_m3.r + alter_sh_p3.r + alter_sh_p2.r + alter_sh_p1.r + 
                             alter_sh.r + alter_sh_m1.r + alter_sh_m2.r + alter_sh_m3.r + 
                             alter_bgmr_p3.r + alter_bgmr_p2.r + alter_bgmr_p1.r + 
                             alter_bgmr.r + alter_bgmr_m1.r + alter_bgmr_m2.r + alter_bgmr_m3.r | 
                             county_fips + ds | 0 | state_abbv, 
                             panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_hd_aitt_sg <- summarize.felm.fit.with.adj(fm = ll_DID.mp.aITT.sg_hd, conley.selector.matrix, 
                                                 panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_DID.mp.aITT.sg_rstu <- felm(asinh_fb_rnstu.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                               stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                               stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                               ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                               geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                               geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                               geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                               geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                               geo_alter_bgmr_m3.r + alter_sh_p3.r + alter_sh_p2.r + alter_sh_p1.r + 
                               alter_sh.r + alter_sh_m1.r + alter_sh_m2.r + alter_sh_m3.r + 
                               alter_bgmr_p3.r + alter_bgmr_p2.r + alter_bgmr_p1.r + 
                               alter_bgmr.r + alter_bgmr_m1.r + alter_bgmr_m2.r + alter_bgmr_m3.r | 
                               county_fips + ds | 0 | state_abbv, 
                               panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_rstu_aitt_sg <- summarize.felm.fit.with.adj(fm = ll_DID.mp.aITT.sg_rstu, conley.selector.matrix, 
                                                   panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_DID.mp.aITT.sg_cbgv <- felm(log_sg_cbgv.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                               stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                               stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                               ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                               geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                               geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                               geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                               geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                               geo_alter_bgmr_m3.r + alter_sh_p3.r + alter_sh_p2.r + alter_sh_p1.r + 
                               alter_sh.r + alter_sh_m1.r + alter_sh_m2.r + alter_sh_m3.r + 
                               alter_bgmr_p3.r + alter_bgmr_p2.r + alter_bgmr_p1.r + 
                               alter_bgmr.r + alter_bgmr_m1.r + alter_bgmr_m2.r + alter_bgmr_m3.r | 
                               county_fips + ds | 0 | state_abbv, 
                               panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_cbgv_aitt_sg <- summarize.felm.fit.with.adj(fm = ll_DID.mp.aITT.sg_cbgv, conley.selector.matrix, 
                                                   panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_DID.mp.aITT.sg_btvrc <- felm(fb_btvrc.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_p3.r + geo_alter_sh_p2.r + geo_alter_sh_p1.r + 
                                geo_alter_sh.r + geo_alter_sh_m1.r + geo_alter_sh_m2.r + geo_alter_sh_m3.r + 
                                geo_alter_bgmr_p3.r + geo_alter_bgmr_p2.r + geo_alter_bgmr_p1.r + 
                                geo_alter_bgmr.r + geo_alter_bgmr_m1.r + geo_alter_bgmr_m2.r + 
                                geo_alter_bgmr_m3.r + alter_sh_p3.r + alter_sh_p2.r + alter_sh_p1.r + 
                                alter_sh.r + alter_sh_m1.r + alter_sh_m2.r + alter_sh_m3.r + 
                                alter_bgmr_p3.r + alter_bgmr_p2.r + alter_bgmr_p1.r + 
                                alter_bgmr.r + alter_bgmr_m1.r + alter_bgmr_m2.r + alter_bgmr_m3.r | 
                                county_fips + ds | 0 | state_abbv, 
                                panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_btvrc_aitt_sg <- summarize.felm.fit.with.adj(fm = ll_DID.mp.aITT.sg_btvrc, conley.selector.matrix, 
                                                    panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))


Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [20]:
# DiD - alter policy ITT just geo (split in- and out of state) (with leads and lags)

ll_did_sgnhd_geo_split <-  felm(asinh_sg_nhd.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_sgnhd_geo_split <- summarize.felm.fit.with.adj(fm = ll_did_sgnhd_geo_split, conley.selector.matrix, 
                                                      panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_did_fbrnstu_geo_split <-  felm(asinh_fb_rnstu.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_fbrnstu_geo_split <- summarize.felm.fit.with.adj(fm = ll_did_fbrnstu_geo_split, conley.selector.matrix, 
                                                        panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_did_sgcbgv_geo_split <-  felm(log_sg_cbgv.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_sgcbgv_geo_split <- summarize.felm.fit.with.adj(fm = ll_did_sgcbgv_geo_split, conley.selector.matrix, 
                                                       panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_did_btvrc_geo_split <-  felm(fb_btvrc.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_btvrc_geo_split <- summarize.felm.fit.with.adj(fm = ll_did_btvrc_geo_split, conley.selector.matrix, 
                                                      panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”
Warning message in sqrt(solve(R %*% V %*% t(R))):
“NaNs produced”


In [21]:
ll_did_sgnhd_both_split <-  felm(asinh_sg_nhd.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r +          
                                alter_sh_same_state_p3.r + alter_sh_same_state_p2.r + 
                                alter_sh_same_state_p1.r + alter_sh_same_state.r + 
                                alter_sh_same_state_m1.r + alter_sh_same_state_m2.r + 
                                alter_sh_same_state_m3.r + alter_ban_gmr_same_state_p3.r + 
                                alter_ban_gmr_same_state_p2.r + alter_ban_gmr_same_state_p1.r + 
                                alter_ban_gmr_same_state.r + alter_ban_gmr_same_state_m1.r +
                                alter_ban_gmr_same_state_m2.r + alter_ban_gmr_same_state_m3.r +
                                alter_sh_diff_state_p3.r + alter_sh_diff_state_p2.r + 
                                alter_sh_diff_state_p1.r + alter_sh_diff_state.r + 
                                alter_sh_diff_state_m1.r + alter_sh_diff_state_m2.r + 
                                alter_sh_diff_state_m3.r + alter_ban_gmr_diff_state_p3.r + 
                                alter_ban_gmr_diff_state_p2.r + alter_ban_gmr_diff_state_p1.r + 
                                alter_ban_gmr_diff_state.r + alter_ban_gmr_diff_state_m1.r +
                                alter_ban_gmr_diff_state_m2.r + alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_sgnhd_both_split <- summarize.felm.fit.with.adj(fm = ll_did_sgnhd_both_split, conley.selector.matrix, 
                                                       panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_did_fbrnstu_both_split <-  felm(asinh_fb_rnstu.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r +          
                                alter_sh_same_state_p3.r + alter_sh_same_state_p2.r + 
                                alter_sh_same_state_p1.r + alter_sh_same_state.r + 
                                alter_sh_same_state_m1.r + alter_sh_same_state_m2.r + 
                                alter_sh_same_state_m3.r + alter_ban_gmr_same_state_p3.r + 
                                alter_ban_gmr_same_state_p2.r + alter_ban_gmr_same_state_p1.r + 
                                alter_ban_gmr_same_state.r + alter_ban_gmr_same_state_m1.r +
                                alter_ban_gmr_same_state_m2.r + alter_ban_gmr_same_state_m3.r +
                                alter_sh_diff_state_p3.r + alter_sh_diff_state_p2.r + 
                                alter_sh_diff_state_p1.r + alter_sh_diff_state.r + 
                                alter_sh_diff_state_m1.r + alter_sh_diff_state_m2.r + 
                                alter_sh_diff_state_m3.r + alter_ban_gmr_diff_state_p3.r + 
                                alter_ban_gmr_diff_state_p2.r + alter_ban_gmr_diff_state_p1.r + 
                                alter_ban_gmr_diff_state.r + alter_ban_gmr_diff_state_m1.r +
                                alter_ban_gmr_diff_state_m2.r + alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_fbrnstu_both_split <- summarize.felm.fit.with.adj(fm = ll_did_fbrnstu_both_split, conley.selector.matrix, 
                                                         panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_did_sgcbgv_both_split <-  felm(log_sg_cbgv.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r +          
                                alter_sh_same_state_p3.r + alter_sh_same_state_p2.r + 
                                alter_sh_same_state_p1.r + alter_sh_same_state.r + 
                                alter_sh_same_state_m1.r + alter_sh_same_state_m2.r + 
                                alter_sh_same_state_m3.r + alter_ban_gmr_same_state_p3.r + 
                                alter_ban_gmr_same_state_p2.r + alter_ban_gmr_same_state_p1.r + 
                                alter_ban_gmr_same_state.r + alter_ban_gmr_same_state_m1.r +
                                alter_ban_gmr_same_state_m2.r + alter_ban_gmr_same_state_m3.r +
                                alter_sh_diff_state_p3.r + alter_sh_diff_state_p2.r + 
                                alter_sh_diff_state_p1.r + alter_sh_diff_state.r + 
                                alter_sh_diff_state_m1.r + alter_sh_diff_state_m2.r + 
                                alter_sh_diff_state_m3.r + alter_ban_gmr_diff_state_p3.r + 
                                alter_ban_gmr_diff_state_p2.r + alter_ban_gmr_diff_state_p1.r + 
                                alter_ban_gmr_diff_state.r + alter_ban_gmr_diff_state_m1.r +
                                alter_ban_gmr_diff_state_m2.r + alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_sgcbgv_both_split <- summarize.felm.fit.with.adj(fm = ll_did_sgcbgv_both_split, conley.selector.matrix, 
                                                        panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

ll_did_btvrc_both_split <-  felm(fb_btvrc.r ~ 0 + stay_home_p3.r + stay_home_p2.r + 
                                stay_home_p1.r + stay_home.r + stay_home_m1.r + stay_home_m2.r + 
                                stay_home_m3.r + ban_gmr_p3.r + ban_gmr_p2.r + ban_gmr_p1.r + 
                                ban_gmr.r + ban_gmr_m1.r + ban_gmr_m2.r + ban_gmr_m3.r + 
                                geo_alter_sh_same_state_p3.r + geo_alter_sh_same_state_p2.r + 
                                geo_alter_sh_same_state_p1.r + geo_alter_sh_same_state.r + 
                                geo_alter_sh_same_state_m1.r + geo_alter_sh_same_state_m2.r + 
                                geo_alter_sh_same_state_m3.r + geo_alter_ban_gmr_same_state_p3.r + 
                                geo_alter_ban_gmr_same_state_p2.r + geo_alter_ban_gmr_same_state_p1.r + 
                                geo_alter_ban_gmr_same_state.r + geo_alter_ban_gmr_same_state_m1.r +
                                geo_alter_ban_gmr_same_state_m2.r + geo_alter_ban_gmr_same_state_m3.r +
                                geo_alter_sh_diff_state_p3.r + geo_alter_sh_diff_state_p2.r + 
                                geo_alter_sh_diff_state_p1.r + geo_alter_sh_diff_state.r + 
                                geo_alter_sh_diff_state_m1.r + geo_alter_sh_diff_state_m2.r + 
                                geo_alter_sh_diff_state_m3.r + geo_alter_ban_gmr_diff_state_p3.r + 
                                geo_alter_ban_gmr_diff_state_p2.r + geo_alter_ban_gmr_diff_state_p1.r + 
                                geo_alter_ban_gmr_diff_state.r + geo_alter_ban_gmr_diff_state_m1.r +
                                geo_alter_ban_gmr_diff_state_m2.r + geo_alter_ban_gmr_diff_state_m3.r +          
                                alter_sh_same_state_p3.r + alter_sh_same_state_p2.r + 
                                alter_sh_same_state_p1.r + alter_sh_same_state.r + 
                                alter_sh_same_state_m1.r + alter_sh_same_state_m2.r + 
                                alter_sh_same_state_m3.r + alter_ban_gmr_same_state_p3.r + 
                                alter_ban_gmr_same_state_p2.r + alter_ban_gmr_same_state_p1.r + 
                                alter_ban_gmr_same_state.r + alter_ban_gmr_same_state_m1.r +
                                alter_ban_gmr_same_state_m2.r + alter_ban_gmr_same_state_m3.r +
                                alter_sh_diff_state_p3.r + alter_sh_diff_state_p2.r + 
                                alter_sh_diff_state_p1.r + alter_sh_diff_state.r + 
                                alter_sh_diff_state_m1.r + alter_sh_diff_state_m2.r + 
                                alter_sh_diff_state_m3.r + alter_ban_gmr_diff_state_p3.r + 
                                alter_ban_gmr_diff_state_p2.r + alter_ban_gmr_diff_state_p1.r + 
                                alter_ban_gmr_diff_state.r + alter_ban_gmr_diff_state_m1.r +
                                alter_ban_gmr_diff_state_m2.r + alter_ban_gmr_diff_state_m3.r | 
                             county_fips + ds | 0 | 0, 
                          panel_ll, keepCX = T, weights = panel_ll$n)
ll_out_btvrc_both_split <- summarize.felm.fit.with.adj(fm = ll_did_btvrc_both_split, conley.selector.matrix, 
                                                       panel_ll$state_abbv, n_ds=length(unique(panel_ll$ds)))

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or indefinite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or indefinite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or indefinite”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or indefinite”


In [22]:
model_save_location = '/nfs/sloanlab004/projects/covid_mobility_proj/models/'

# Save focal only models with leads and lags
saveRDS(ll_DiD.sp_fbrstu, file=str_c(model_save_location, 'll_did_sgnhd_focal.rds'))
saveRDS(ll_out_fbrstu, file=str_c(model_save_location, 'll_did_sgnhd_focal_vcov.rds'))
saveRDS(ll_DiD.sp_hd, file=str_c(model_save_location, 'll_did_fbnrstu_focal.rds'))
saveRDS(ll_out_sp_hd, file=str_c(model_save_location, 'll_did_fbrnstu_focal_vcov.rds'))
saveRDS(ll_DiD.sp_cbgv, file=str_c(model_save_location, 'll_did_sgcbgv_focal.rds'))
saveRDS(ll_out_sp_cbgv, file=str_c(model_save_location, 'll_did_sgcbgv_focal_vcov.rds'))
saveRDS(ll_DiD.sp_btvrc, file=str_c(model_save_location, 'll_did_fbbtvrc_focal.rds'))
saveRDS(ll_out_sp_btvrc, file=str_c(model_save_location, 'll_did_fbbtvrc_focal_vcov.rds'))

# Save focal + geo models with leads and lags
saveRDS(ll_DID.mp_go.aITT.sg_hd, file=str_c(model_save_location, 'll_did_sgnhd_geo.rds'))
saveRDS(ll_out_hd_aitt_sg_go, file=str_c(model_save_location, 'll_did_sgnhd_geo_vcov.rds'))
saveRDS(ll_DID.mp_go.aITT.sg_rstu, file=str_c(model_save_location, 'll_did_fbnrstu_geo.rds'))
saveRDS(ll_out_rstu_aitt_sg_go, file=str_c(model_save_location, 'll_did_fbrnstu_geo_vcov.rds'))
saveRDS(ll_DID.mp_go.aITT.sg_cbgv, file=str_c(model_save_location, 'll_did_sgcbgv_geo.rds'))
saveRDS(ll_out_cbgv_aitt_sg_go, file=str_c(model_save_location, 'll_did_sgcbgv_geo_vcov.rds'))
saveRDS(ll_DID.mp_go.aITT.sg_btvrc, file=str_c(model_save_location, 'll_did_fbbtvrc_geo.rds'))
saveRDS(ll_out_btvrc_aitt_sg_go, file=str_c(model_save_location, 'll_did_fbbtvrc_geo_vcov.rds'))

# Save focal + geo + social models with leads and lags
saveRDS(ll_DID.mp.aITT.sg_hd, file=str_c(model_save_location, 'll_did_sgnhd_both.rds'))  
saveRDS(ll_out_hd_aitt_sg, file=str_c(model_save_location, 'll_did_sgnhd_both_vcov.rds'))
saveRDS(ll_DID.mp.aITT.sg_rstu, file=str_c(model_save_location, 'll_did_fbnrstu_both.rds'))                                      
saveRDS(ll_out_rstu_aitt_sg, file=str_c(model_save_location, 'll_did_fbrnstu_both_vcov.rds'))
saveRDS(ll_DID.mp.aITT.sg_cbgv, file=str_c(model_save_location, 'll_did_sgcbgv_both.rds'))                 
saveRDS(ll_out_cbgv_aitt_sg, file=str_c(model_save_location, 'll_did_sgcbgv_both_vcov.rds'))
saveRDS(ll_DID.mp.aITT.sg_btvrc, file=str_c(model_save_location, 'll_did_fbbtvrc_both.rds'))          
saveRDS(ll_out_btvrc_aitt_sg, file=str_c(model_save_location, 'll_did_fbbtvrc_both_vcov.rds'))

# Save focal + geo split models with leads and lags
saveRDS(ll_did_sgnhd_geo_split, file=str_c(model_save_location, 'll_did_sgnhd_geo_split.rds'))
saveRDS(ll_out_sgnhd_geo_split, file=str_c(model_save_location, 'll_out_sgnhd_geo_split.rds'))
saveRDS(ll_did_fbrnstu_geo_split, file=str_c(model_save_location, 'll_did_fbrnstu_geo_split.rds'))
saveRDS(ll_out_fbrnstu_geo_split, file=str_c(model_save_location, 'll_out_fbrnstu_geo_split.rds'))
saveRDS(ll_did_sgcbgv_geo_split, file=str_c(model_save_location, 'll_did_sgcbgv_geo_split.rds'))
saveRDS(ll_out_sgcbgv_geo_split, file=str_c(model_save_location, 'll_out_sgcbgv_geo_split.rds'))
saveRDS(ll_did_btvrc_geo_split, file=str_c(model_save_location, 'll_did_btvrc_geo_split.rds'))
saveRDS(ll_out_btvrc_geo_split, file=str_c(model_save_location, 'll_out_btvrc_geo_split.rds'))

# Save focal + geo + alter split models with leads and lags
saveRDS(ll_did_sgnhd_both_split, file=str_c(model_save_location, 'll_did_sgnhd_both_split.rds'))                       
saveRDS(ll_out_sgnhd_both_split, file=str_c(model_save_location, 'll_out_sgnhd_both_split.rds'))
saveRDS(ll_did_fbrnstu_both_split, file=str_c(model_save_location, 'll_did_fbrnstu_both_split.rds'))                          
saveRDS(ll_out_fbrnstu_both_split, file=str_c(model_save_location, 'll_out_fbrnstu_both_split.rds'))
saveRDS(ll_did_sgcbgv_both_split, file=str_c(model_save_location, 'll_did_sgcbgv_both_split.rds'))                        
saveRDS(ll_out_sgcbgv_both_split, file=str_c(model_save_location, 'll_out_sgcbgv_both_split.rds'))
saveRDS(ll_did_btvrc_both_split, file=str_c(model_save_location, 'll_did_btvrc_both_split.rds'))
saveRDS(ll_out_btvrc_both_split, file=str_c(model_save_location, 'll_out_btvrc_both_split.rds'))                                                      

In [23]:
# Understand lack of power in interacted model

#DID.mp.aITT.sg_hd_int <- felm(asinh_sg_nhd.r ~ 0 +  ban_gmr.r*(alter_bgmr.r + geo_alter_bgmr.r) +
#                          stay_home.r*(alter_sh.r + geo_alter_sh.r) | county_fips + ds | 0 | state_abbv, 
#                          panel, keepCX = T, weights = panel$n)
panel["ban_gmr.r:alter_bgmr.r"] <- panel["ban_gmr.r"] * panel["alter_bgmr.r"]
panel["ban_gmr.r:geo_alter_bgmr.r"] <- panel["ban_gmr.r"] * panel["geo_alter_bgmr.r"]
panel["stay_home.r:alter_sh.r"] <- panel["stay_home.r"] * panel["alter_sh.r"]
panel["stay_home.r:geo_alter_sh.r"] <- panel["stay_home.r"] * panel["geo_alter_sh.r"]
cor(panel[c("ban_gmr.r", "alter_bgmr.r", "geo_alter_bgmr.r", "stay_home.r", "alter_sh.r", "geo_alter_sh.r",
           "ban_gmr.r:alter_bgmr.r", "ban_gmr.r:geo_alter_bgmr.r", "stay_home.r:alter_sh.r",
           "stay_home.r:geo_alter_sh.r")])

,ban_gmr.r,alter_bgmr.r,geo_alter_bgmr.r,stay_home.r,alter_sh.r,geo_alter_sh.r,ban_gmr.r:alter_bgmr.r,ban_gmr.r:geo_alter_bgmr.r,stay_home.r:alter_sh.r,stay_home.r:geo_alter_sh.r
ban_gmr.r,1.00000000,0.92526634,0.8115770,-0.82874076,-0.76178091,-0.63791634,0.09527268,0.07059460,0.12350919,0.05198972
alter_bgmr.r,0.92526634,1.00000000,0.9104655,-0.76057084,-0.82540765,-0.72592677,0.10474705,0.09629571,0.14404479,0.09797359
geo_alter_bgmr.r,0.81157703,0.91046546,1.0000000,-0.64208927,-0.73042890,-0.79720258,0.10275972,0.13217515,0.10230406,0.11539397
stay_home.r,-0.82874076,-0.76057084,-0.6420893,1.00000000,0.91828178,0.77746802,-0.15154247,-0.09756294,-0.07149080,0.00907932
alter_sh.r,-0.76178091,-0.82540765,-0.7304289,0.91828178,1.00000000,0.89146856,-0.16796830,-0.13633426,-0.09230494,-0.04464791
geo_alter_sh.r,-0.63791634,-0.72592677,-0.7972026,0.77746802,0.89146856,1.00000000,-0.14786687,-0.17349311,-0.04973430,-0.07591373
ban_gmr.r:alter_bgmr.r,0.09527268,0.10474705,0.1027597,-0.15154247,-0.16796830,-0.14786687,1.00000000,0.91100337,0.58851949,0.51277514
ban_gmr.r:geo_alter_bgmr.r,0.07059460,0.09629571,0.1321751,-0.09756294,-0.13633426,-0.17349311,0.91100337,1.00000000,0.51086905,0.58839479
stay_home.r:alter_sh.r,0.12350919,0.14404479,0.1023041,-0.07149080,-0.09230494,-0.04973430,0.58851949,0.51086905,1.00000000,0.87777033
stay_home.r:geo_alter_sh.r,0.05198972,0.09797359,0.1153940,0.00907932,-0.04464791,-0.07591373,0.51277514,0.58839479,0.87777033,1.00000000


In [24]:
# Below this is scrap from trying to do policy contrasts using Dean's old code. Keeping for now just in case.

In [25]:
# # Policy counterfactuals...
# 
# # Same state, no spillovers
# source('~/covid_mobility/paper_code/utils/contrasts.r')
# pc_same_state_no_spillovers <- policy_contrast_linear_many_models(
#     list(DiD.sp_hd, DiD.sp_cbgv,
#          DiD.sp_fbrstu, DiD.sp_btvrc),
#     panel %>% mutate(
#         stay_home.r = 1
#     ),
#     panel %>% mutate(
#         stay_home.r = 0
#     ),
#     weights = panel$n,
#     vcv = list(out_sp_hd$vcov.both, out_sp_cbgv$vcov.both,
#               out_fbrstu$vcov.both, out_sp_btvrc$vcov.both)
# ) %>% 
# mutate(counterfactual = 'Shelter-in-place in own state',
#        model = 'No spillovers')

In [26]:
#pc_same_state_no_spillovers